This notebook shows how to run a simple 2D example of DIVA.<br>
The input files and the figures are created within the notebook.<br>
The reading/writing of input files is done through specific classes.

In [1]:
import os
import logging
import shutil
import subprocess
import numpy as np
import importlib
import diva_classes

/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
importlib.reload(diva_classes)

<module 'diva_classes' from '/home/ctroupin/Projects/SeaDataCloud/Python/diva_classes.py'>

# Setup 

Adapt the path to the diva installation you want to work with.

In [3]:
divadir = "/home/ctroupin/Software/DIVA/diva-4.7.1"

## Logging configuration

Logging already configured in the main class. 

In [4]:
#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)

## Directories

Create the directory and in the input files names.

In [5]:
DivaDirs = diva_classes.DivaDirectory(divadir)

INFO:diva2D:Diva binary directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/bin
INFO:diva2D:Diva source directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/src/Fortan
INFO:diva2D:Diva 2D directory: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped
INFO:diva2D:Diva 4D directory: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology
INFO:diva2D:Diva 4D input directory: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/input


In [ ]:
DivaFile = diva_classes.Diva2Dfiles(DivaDirs.diva2ddir)

# Prepare the input files

## Contour

Create a sequence of points representing the contour (no crossing, no duplicate points, etc).<br>
Each coordinate (lon, lat) is defined as a [ndarray](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html).

In [ ]:
lon = np.array([[0., 2., 2., 0.], [2.5, 6., 6., 2.5]])
lat = np.array([[0., 0., 4., 4.], [0., 0., 3., 3.]]) 

Create a **contour** object: 

In [ ]:
contour2D = diva_classes.Diva2DContours(lon, lat)

and write its content to a file:

In [ ]:
contour2D.write_to_file(DivaFile.contourfile)

## Data

Create arrays of coordinates, values and optionnaly weights representing the data points.<br>
In this example we assume that all the data points have an identical weights.

In [ ]:
x = np.array((1.2, 2., 3., 5.5))  
y = np.array((2.1, 3.5, 2., 0.2))
z = np.array((1., -2., 4, -1))

In [ ]:
data2D = diva_classes.Diva2DData(x, y, z)

that we write to a file:

In [ ]:
data2D.write_to_file(DivaFile.datafile)

## Parameters

We need to set the parameters defining the domain and the analysis parameters.

In [ ]:
xmin, ymin, nx, ny, dx, dy = 0.0, 0.0, 61, 81, 0.1, 0.05

In [ ]:
CorrelationLength = 1.5
SignalToNoiseRatio = 0.5
VarianceBackgroundField = 1.0
ExclusionValue = -999.
iCoordChange = 0
iSpec = 11
iReg = 0

We create an object that stores all the parameter values:

In [ ]:
parameters2D = diva_classes.Diva2DParameters(CorrelationLength, iCoordChange, iSpec, iReg, xmin, ymin, dx, dy, nx, ny,
                                ExclusionValue, SignalToNoiseRatio, VarianceBackgroundField)

and we write them into a file:

In [ ]:
parameters2D.write_to_file(DivaFile.parameterfile)

# Run Diva

Now that the input files have been created we can perform a Diva analysis.

## Mesh generation

In [ ]:
logging.info("Generating finite element mesh")
subprocess.Popen("./divamesh", cwd=DivaDirs.diva2ddir, shell=True)

## Run the diva calculation

In [ ]:
logging.info("Generating gridded field")
subprocess.Popen("./divacalc", cwd=DivaDirs.diva2ddir, shell=True)

# Time for some plots

In [ ]:
import netCDF4
import matplotlib.pyplot as plt
%matplotlib inline

## Input files

Let's represent the osbervations, the grid and the contours.

In [ ]:
fig = plt.figure(figsize=(2, 2))
contour2D.add_to_plot(linewidth=2)
data2D.add_to_plot(s=10)
parameters2D.plot_outputgrid()

## Results 

The results are stored in a netCDF file that we can access using the resultfile property of DivaFile.<br>
The field is then added to the plot. In addition we can include the locations of the observations.

In [ ]:
fig = plt.figure(figsize=(2, 2))
results2D = diva_classes.Diva2DResults(DivaFile.resultfile)
results2D.add_to_plot('result')
data2D.add_to_plot(cmap=plt.cm.viridis, s=5)

The error field is represented using a similar command:

In [ ]:
fig = plt.figure(figsize=(2, 2))
results2D.add_to_plot('error', cmap=plt.cm.hot_r)
data2D.add_positions_to_plot(s=3)

In [ ]:
importlib.reload(diva_classes)

## Finite-element mesh

In [ ]:
mesh2D = diva_classes.Diva2DMesh(DivaFile.meshfile, DivaFile.meshtopofile)

In [ ]:
fig = plt.figure(figsize=(2, 2))
ax = fig.add_subplot(111)
mesh2D.add_to_plot(ax, lw=0.1)